In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from firebase import FirebaseManager 
from selenium.webdriver.common.keys import Keys

In [4]:
def retrieve_professions_from_field_first(field):
    h3_elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Professions')]")
    print(h3_elements)
    if len(h3_elements)==0:
        raise Exception(f"Aucun element trouvé pour la première fonction pour {field}")
    for h3_element in h3_elements:
        ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")
        driver.execute_script("arguments[0].scrollIntoView(true);", h3_element)
        if ul_element:
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    datas[field].append(text)
        time.sleep(0.5)
    # driver.back()



def retrieve_professions_from_field_second(field):
    strong_elements = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Professions')]]")
    print(f"strong: {strong_elements}")
    if len(strong_elements)==0:
        errors.append(field)
        raise Exception(f"Aucun element trouvé pour la deuxième fonction pour {field}")
    for strong_element in strong_elements:
        ul_element = strong_element.find_element(By.XPATH, "following-sibling::ul")
        if ul_element:
            driver.execute_script("arguments[0].scrollIntoView(true);", ul_element)
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    datas[field].append(text)
        time.sleep(0.5)
    # driver.back()

def retrieve_professions_from_field(field):
    global nb_of_functions_tried, nb_of_functions
    try:
        if nb_of_functions_tried==1:
            retrieve_professions_from_field_first(field)
            driver.back()
        if nb_of_functions_tried==2:
            retrieve_professions_from_field_second(field)
            driver.back()
            nb_of_functions_tried=1
    except Exception as e:
        print(e)
        nb_of_functions_tried+=1
        retrieve_professions_from_field(field)

    if nb_of_functions_tried > nb_of_functions:
        driver.back()
        nb_of_functions_tried=1
    return



In [16]:
driver = webdriver.Chrome()
driver.get("https://www.ulaval.ca/etudes/programmes")
title = driver.title

# Keys represente fields and values are arrays whose contains professions 
datas={}
errors=[]
nb_of_functions=2
nb_of_functions_tried=1

pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
print(pagination_next_element)
while pagination_next_element:
    results_element= driver.find_element(By.ID,"resultats")
    children=results_element.find_elements(By.XPATH, "./ul[@class='views-row']")

    for child in children:
        element=child.find_element(By.XPATH, "./li/a")
        field=element.find_element(By.XPATH,"./div[@class='main-info']/h3[@class='titre-programme']").text
        datas[field]=[]
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        retrieve_professions_from_field(field)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", pagination_next_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", pagination_next_element.find_element(By.TAG_NAME, "a"))
    try:
        pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
    except Exception as e:
        pagination_next_element=None
   

print(datas)


<selenium.webdriver.remote.webelement.WebElement (session="f1691872b3cf914cbe632978d2bce6ed", element="f.BAF25372A6E8297D67E7D5D87E0907AA.d.EAB6BC0227139353B1E84151048E046F.e.78")>


NameError: name 'retrieve_professions_from_field' is not defined

In [2]:
collection_names=["professions_by_field","posts_by_profession"]
credential_path="./indeedrecommender-firebase-adminsdk-3zci4-966775593c.json"
firebase_manager=FirebaseManager(credential_path)
for key in datas.keys():
    firebase_manager.add_data_to_firestore(collection_names[0],key,datas[key])


Data added to professions_by_field/key


In [33]:
driver = webdriver.Chrome()
def indeedDataFill(ulavalPosition, location):
    driver.get("https://emplois.ca.indeed.com/")
    title_input = driver.find_element(By.XPATH, "//input[@id='text-input-what']")
    location_input = driver.find_element(By.XPATH, "//input[@id='text-input-where']")
    location_input.clear()
    location_input.send_keys(location)
    title_input.send_keys(ulavalPosition, Keys.RETURN)

    return title_input, location_input
    

In [58]:
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

def stateDateToDate(stateDate):
   date=None
   for str in stateDate.split(" "):
      try:
         date= datetime.strptime(str, '%Y-%m-%d')
      except ValueError:
         continue
      if bool(re.compile(r'\b(\w*\+\w*)\b', re.IGNORECASE).search(str)):
         str= str.replace("+", "")
      if str.isnumeric():
            if bool(re.compile(r'\b(\w*jour\w*|\w*day\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(days=int(str)))
            if bool(re.compile(r'\b(\w*mois\w*|\w*month\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(months=int(str)))
            if bool(re.compile(r'\b(\w*an \w*|\w*year\w*|\w*année\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(years=int(str)))
            
   return date

In [59]:
positionsDetailsList = {}

def indeedPositionInfos(profession):
    global positionsDetailsList
    positionsDescriptionList = driver.find_element(By.XPATH, "//div[@id='mosaic-provider-jobcards']")
    
    ulElement = positionsDescriptionList.find_element(By.XPATH, "./ul")
    liElements = ulElement.find_elements(By.XPATH, "./li[@class='css-5lfssm eu4oa1w0']")
    for li in liElements:
        positionsDetails = {}
        driver.execute_script("arguments[0].scrollIntoView(true);", li)
        time.sleep(0.5)
        try:
          position=li.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]/a")
          positionTitle = driver.execute_script("return arguments[0].innerText;", position)
          print(positionTitle)
        except Exception as e:
            print("No position found")
            position=None

        try:
          dateElement=li.find_element(By.XPATH, ".//span[@data-testid='myJobsStateDate']")
          positionPostedDate=stateDateToDate(driver.execute_script("return arguments[0].innerText;", dateElement))
        except Exception as e:
            print("No Date found")
            dateElement=None
        
        if position :
            driver.execute_script("arguments[0].click();", position)
            time.sleep(0.5)
            try:
               positionsDetailsElement = driver.find_element(By.XPATH, "//div[@id='jobsearch-ViewjobPaneWrapper']")
            except Exception as e:
               positionsDetails=None
               print("No position details found")
               
            
            if positionsDetailsElement:	
                try:
                    companyName = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyName']"))
                except Exception as e:
                    print("No company name found")
                    companyName=None
                try:
                    location = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyLocation']"))
                except Exception as e:  
                    print("No location found")
                    location=None
                try:
                    jobType = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@id='salaryInfoAndJobType]']/span"))
                except Exception as e:
                    print("No job type found")
                    jobType=None
                try:
                    jobType = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Permanent-tile]']"))
                except Exception as e:
                    print("No job type found")
                    jobType=""
                try:
                    jobType1 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Full-time-tile]']"))
                except Exception as e:
                    print("No job type found")
                    jobType1=""
                jobType = jobType + jobType1
                try:
                    shiftAndSchedule = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Monday to Friday-tile']"))
                except Exception as e:
                    print("No shift and schedule found")
                    shiftAndSchedule=None
                jobBenefits = []
                try:
                    benefits = positionsDetailsElement.find_elements(By.XPATH, ".//div[@id='benefits']/ul[@class='css-8tnble eu4oa1w0']/li")
                    for benefit in benefits:
                        jobBenefits.append(driver.execute_script("return arguments[0].innerText;",benefit))
                except Exception as e:
                    print("No benefits found")
                    benefits=None
                    
                try:
                    description = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@id='jobDescriptionText']"))
                except Exception as e:
                    print("No description found")
                    description=None
        
        positionsDetails["positionTitle"]=positionTitle
        positionsDetails["companyName"]=companyName
        positionsDetails["location"]=location
        positionsDetails["jobType"]=jobType
        positionsDetails["description"]=description
        positionsDetails["datePosted"]=positionPostedDate
        positionsDetails["salary"]=description
        positionsDetails["shiftAndSchedule"]=shiftAndSchedule
        positionsDetails["jobBenefits"]=jobBenefits
        if profession not in positionsDetailsList:
            positionsDetailsList[profession]=[]
        positionsDetailsList[profession].append(positionsDetails)

In [60]:
indeedDataFill("Voleur", "Canada")
try:
    pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li[@data-testid='pagination-page-next']/a")
except Exception as e:
    pagination_next_element=None

print(pagination_next_element)
if not pagination_next_element:
    indeedPositionInfos("Actuaire")
if pagination_next_element:
    while pagination_next_element:
        indeedPositionInfos("Actuaire")
        try:
            pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li[@data-testid='pagination-page-next']/a")
        except Exception as e:
            pagination_next_element=None


None
Tuteur(trice) en français ou mathématiques
No job type found
No job type found
No job type found
No shift and schedule found
PLOMBIER
No job type found
No job type found
No job type found
No shift and schedule found
Brasseur
No job type found
No job type found
No job type found
No shift and schedule found
CONTRÔLEUR FINANCIER ADJOINT
No job type found
No job type found
No job type found
No shift and schedule found
Directeur(trice) adjoint(e)
No job type found
No job type found
No job type found
No shift and schedule found
No position found
No Date found
Directeur(trice) commercial (e)
No job type found
No job type found
No job type found
No shift and schedule found
Répartiteur de vol
No job type found
No job type found
No job type found
No shift and schedule found
Répartiteur de vol
No job type found
No job type found
No job type found
No shift and schedule found
ramasseur/ramasseuse de poulets
No job type found
No job type found
No job type found
No shift and schedule found
Contr

In [56]:
print(positionsDetailsList)

{'Actuaire': [{'positionTitle': 'Conseiller en actuariat', 'companyName': 'Beneva', 'location': 'Ville de Québec, QC•Travail hybride', 'jobType': '', 'description': "Titre d'emploi:\nConseiller en actuariat\nStatut:\nRégulier\nSommaire:\nVous souhaitez travailler au sein d’une équipe innovante composée d’experts aux connaissances multiples et jouer un rôle important dans l’amélioration de la performance et des solutions de la vice-présidence, vous êtes le candidat que nous recherchons pour se joindre à nous!\nÀ titre de conseiller en actuariat, vous ferez partie de l’équipe en charge d’appuyer les développements de la vice-présidence actuariat corporatif qui est responsable de l’évaluation des engagements envers les assurés, des ratios d’expérience des produits, de l’évaluation et de l’évolution de la santé financière ainsi que de la réassurance pour toutes les lignes d’affaires.\nAinsi, vous contribuerez au mandat de la direction - performance, données et solutions actuarielles en enc

In [70]:
positionsDetailsList["Actuaire"][5]

{'positionTitle': 'Directeur(trice) adjoint(e)',
 'companyName': 'Betterprof',
 'location': 'Ville de Québec, QC',
 'jobType': '',
 'description': "Betterprof est à la recherche d'un(e) tuteur(trice) scolaire passionné(e) à Québec pour rejoindre notre équipe dynamique. Si vous avez une passion pour l'enseignement, que vous excellez en français et en mathématiques, et que vous souhaitez aider les étudiants à atteindre leur plein potentiel, alors cette opportunité est faite pour vous.\n\nDescription du poste :\n\nEn tant que tuteur(trice) scolaire en français et mathématiques chez Betterprof, vous serez chargé(e) d'aider les élèves de tous niveaux à développer leurs compétences en français et en mathématiques. Vous travaillerez en étroite collaboration avec nos étudiants pour les aider à comprendre les concepts, à améliorer leurs compétences académiques et à renforcer leur confiance en eux. Vos principales responsabilités seront les suivantes :\n\nResponsabilités :\n\nFournir un soutien 